#Overview

This sample project is a demonstration of a data engineering ETL pipeline with some data analysis using Spark in Databricks.
For this project, we will use the [NHL Game Data](https://www.kaggle.com/martinellis/nhl-game-data) dataset available on Kaggle.

In this notebook, we will use a typical data engineering workflow to process the data in phases:
- *Note: Normally, we would write our dataframes to parquet files or SQL tables, but for demonstration purposes the data in this notebook is being left as in-memory dataframes.*
- Staging: Download the zip file and extract the raw CSV data.
- Data Lake: Read CSV data into Spark dataframes. Normally, we might write these to parquet files for data lake storage.
- Data Warehouse: Transform data into normalized dataframes. Typically, these might be written to tables for further use in analysis.
- Data Mart: Aggregate normalized dataframes into ready-to-use statistics. These might also be written to tables for use by analysts.

#Import Raw Data

We will use the Kaggle API to download and unzip the dataset through bash.

In [0]:
%sh
pip install kaggle
export KAGGLE_USERNAME=$USERNAME
export KAGGLE_KEY=$API_KEY
kaggle datasets download martinellis/nhl-game-data -p /mnt/NHL_Dataset
cd /mnt/NHL_Dataset
unzip nhl-game-data.zip
rm nhl-game-data.zip


  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=2045f21667cf24843ddebddc7f912ee5e257a98ac6adb068ab11158901a8ad6c
  Stored in directory: /root/.cache/pip/wheels/29/da/11/144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggle
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

  0%|          | 0.00/240M [00:00<?, ?B/s]
  2%|▏         | 4.00M/240M [00:00<00:06, 40.6MB/s]
  6%|▌         | 14.0M/240M [00:00<00:03, 75.4MB/s]
  9%|▉         | 22.0M/240M [00:00<00:03, 65.4MB/s]
 12%|█▏        | 29.0M/240M [00:00<00:03, 58.9MB/s]
 15%|█▍        | 35.0M/240M [00:00<00:04, 47.2MB/s]
 17%|█▋        | 41.0M/240M [00:00<00:04, 48.1MB/s]
 20%|██        | 49.0M/240M [00:00<00:04, 47.7MB/s]
 24%|██▎       | 57.0M/240M [00:01<00:03, 55.8MB/s]
 27%|██▋       | 66.0M/240M [00:01<00:02, 63.4MB/s]
 32%|███▏      | 76.0M/240M [00:01<00:02, 73.6MB/s]
 36%|███▌      | 87.0M/240M [00:01<00:01, 8

In [0]:
# Consolidate module imports into one cell.
import pyparsing as pp
import re
import math
from glob import glob
from pyspark.sql.types import *
from pyspark.sql.functions import *


Since we know what the data contains from the Kaggle documentation, we can anticipate the schemas and predefine them to improve efficiency.

We can see that many fields are shared between CSVs. To automate our schema creation, we first create a dictionary of the field names and their types.

Then, we can use a function to extract the headers from each file, cross-reference them against our dictionary, and create the schema.

In [0]:
field_types = {
  '"abbreviation"': 'StringType()',
  '"assists"': 'ByteType()',
  '"away_goals"': 'ByteType()',
  '"away_team_id"': 'ByteType()',
  '"birthCity"': 'StringType()',
  '"birthDate"': 'DateType()',
  '"birthStateProvince"': 'StringType()',
  '"blocked"': 'ByteType()',
  '"blocked"': 'ShortType()',
  '"dateTime"': 'TimestampType()',
  '"date_time_GMT"': 'TimestampType()',
  '"decision"': 'StringType()',
  '"description"': 'StringType()',
  '"emptyNet"': 'BooleanType()',
  '"evenSaves"': 'ShortType()',
  '"evenShotsAgainst"': 'ShortType()',
  '"evenStrengthSavePercentage"': 'FloatType()',
  '"event"': 'StringType()',
  '"evenTimeOnIce"': 'ShortType()',
  '"faceoffTaken"': 'ByteType()',
  '"faceOffWinPercentage"': 'FloatType()',
  '"faceOffWins"': 'ByteType()',
  '"firstName"': 'StringType()',
  '"franchiseId"': 'ByteType()',
  '"gameWinningGoal"': 'BooleanType()',
  '"game_id"': 'IntegerType()',
  '"giveaways"': 'ByteType()',
  '"goals"': 'ByteType()',
  '"goals_away"': 'ByteType()',
  '"goals_home"': 'ByteType()',
  '"head_coach"': 'StringType()',
  '"height"': 'StringType()',
  '"height_cm"': 'FloatType()',
  '"hits"': 'ByteType()',
  '"hits"': 'ShortType()',
  '"HoA"': 'StringType()',
  '"home_goals"': 'ByteType()',
  '"home_rink_side_start"': 'StringType()',
  '"home_team_id"': 'ByteType()',
  '"lastName"': 'StringType()',
  '"link"': 'StringType()',
  '"nationality"': 'StringType()',
  '"official_name"': 'StringType()',
  '"official_type"': 'StringType()',
  '"outcome"': 'StringType()',
  '"penaltyMinutes"': 'ByteType()',
  '"penaltySeverity"': 'StringType()',
  '"period"': 'ByteType()',
  '"periodTime"': 'ShortType()',
  '"periodTimeRemaining"': 'ShortType()',
  '"periodType"': 'StringType()',
  '"pim"': 'ShortType()',
  '"playerType"': 'StringType()',
  '"player_id"': 'IntegerType()',
  '"play_id"': 'StringType()',
  '"plusMinus"': 'ByteType()',
  '"powerPlayAssists"': 'ByteType()',
  '"powerPlayGoals"': 'ByteType()',
  '"powerPlayOpportunities"': 'ByteType()',
  '"powerPlaySavePercentage"': 'FloatType()',
  '"powerPlaySaves"': 'ShortType()',
  '"powerPlayShotsAgainst"': 'ShortType()',
  '"powerPlayTimeOnIce"': 'ShortType()',
  '"primaryPosition"': 'StringType()',
  '"savePercentage"': 'FloatType()',
  '"saves"': 'ShortType()',
  '"season"': 'IntegerType()',
  '"secondaryType"': 'StringType()',
  '"settled_in"': 'StringType()',
  '"shift_end"': 'ShortType()',
  '"shift_start"': 'ShortType()',
  '"shootsCatches"': 'StringType()',
  '"shortHandedAssists"': 'ByteType()',
  '"shortHandedGoals"': 'ByteType()',
  '"shortHandedSaves"': 'ShortType()',
  '"shortHandedShotsAgainst"': 'ShortType()',
  '"shortHandedTimeOnIce"': 'ShortType()',
  '"shortName"': 'StringType()',
  '"shots"': 'ByteType()',
  '"shots"': 'ShortType()',
  '"startRinkSide"': 'StringType()',
  '"strength"': 'StringType()',
  '"st_x"': 'ByteType()',
  '"st_y"': 'ByteType()',
  '"takeaways"': 'ByteType()',
  '"teamName"': 'StringType()',
  '"team_id"': 'ByteType()',
  '"team_id_against"': 'ByteType()',
  '"team_id_for"': 'ByteType()',
  '"timeOnIce"': 'ShortType()',
  '"type"': 'StringType()',
  '"venue"': 'StringType()',
  '"venue_link"': 'StringType()',
  '"venue_time_zone_id"': 'StringType()',
  '"venue_time_zone_offset"': 'ByteType()',
  '"venue_time_zone_tz"': 'StringType()',
  '"weight"': 'ShortType()',
  '"won"': 'BooleanType()',
  '"x"': 'ByteType()',
  '"y"': 'ByteType()'
}

In [0]:
def create_schema_template(filepath):
  filename = re.sub(r".+/(.+)\.csv", r"\1", filepath)
  with open(filepath) as f:
    header = f.readline().rstrip()
  schema = f"{filename}_schema = StructType(["
  headerList = pp.commaSeparatedList.parseString(header).asList()
  for h in headerList:
    schema += f"\n  StructField({h}, {field_types[h]}, True),"
  schema += "\n])"
  return schema

filelist = sorted(glob("/mnt/NHL_Dataset/*.csv"))
for f in filelist:
  create_schema_string = create_schema_template(f)
  print(create_schema_string)
  exec(create_schema_string)


game_schema = StructType([
  StructField("game_id", IntegerType(), True),
  StructField("season", IntegerType(), True),
  StructField("type", StringType(), True),
  StructField("date_time_GMT", TimestampType(), True),
  StructField("away_team_id", ByteType(), True),
  StructField("home_team_id", ByteType(), True),
  StructField("away_goals", ByteType(), True),
  StructField("home_goals", ByteType(), True),
  StructField("outcome", StringType(), True),
  StructField("home_rink_side_start", StringType(), True),
  StructField("venue", StringType(), True),
  StructField("venue_link", StringType(), True),
  StructField("venue_time_zone_id", StringType(), True),
  StructField("venue_time_zone_offset", ByteType(), True),
  StructField("venue_time_zone_tz", StringType(), True),
])
game_goalie_stats_schema = StructType([
  StructField("game_id", IntegerType(), True),
  StructField("player_id", IntegerType(), True),
  StructField("team_id", ByteType(), True),
  StructField("timeOnIce", ShortType

Similarly, we can create a dictionary for our repartition settings and generate the code for importing the CSVs to dataframes.

In [0]:
repartition_settings = {
  'game': '.repartition("home_team_id", "away_team_id")',
  'game_goalie_stats': '.repartition("player_id")',
  'game_goals': '',
  'game_officials': '.repartition("official_type")',
  'game_penalties': '',
  'game_plays': '.repartition("event")',
  'game_plays_players': '.repartition("player_id")',
  'game_scratches': '.repartition("player_id")',
  'game_shifts': '.repartition("player_id")',
  'game_skater_stats': '.repartition("player_id")',
  'game_teams_stats': '.repartition("team_id")',
  'player_info': '.repartition("player_id")',
  'team_info': '.repartition("team_id")'
}

In [0]:
filelist = sorted(glob("/mnt/NHL_Dataset/*.csv"))
for f in filelist:
  filename = re.sub(r".+/(.+)\.csv", r"\1", f)
  create_dataframe_string = f"{filename}_df = spark.read.csv(\"file:{f}\", header=True, schema={filename}_schema){repartition_settings[filename]}"
  print(create_dataframe_string)
  exec(create_dataframe_string)
  

game_df = spark.read.csv("file:/mnt/NHL_Dataset/game.csv", header=True, schema=game_schema).repartition("home_team_id", "away_team_id")
game_goalie_stats_df = spark.read.csv("file:/mnt/NHL_Dataset/game_goalie_stats.csv", header=True, schema=game_goalie_stats_schema).repartition("player_id")
game_goals_df = spark.read.csv("file:/mnt/NHL_Dataset/game_goals.csv", header=True, schema=game_goals_schema)
game_officials_df = spark.read.csv("file:/mnt/NHL_Dataset/game_officials.csv", header=True, schema=game_officials_schema).repartition("official_type")
game_penalties_df = spark.read.csv("file:/mnt/NHL_Dataset/game_penalties.csv", header=True, schema=game_penalties_schema)
game_plays_df = spark.read.csv("file:/mnt/NHL_Dataset/game_plays.csv", header=True, schema=game_plays_schema).repartition("event")
game_plays_players_df = spark.read.csv("file:/mnt/NHL_Dataset/game_plays_players.csv", header=True, schema=game_plays_players_schema).repartition("player_id")
game_scratches_df = spark.read.csv(

#Generate Shot Statistics

Now, we can generate some statistics of interest. Let's take a look at shots taken in the game_plays_df and see what types of shots are most successful.

First, let's see what type of shot events exist.

In [0]:
display(game_plays_df.select("event").distinct())

event
Goal
Stoppage
Shot
Early Intermission End
Emergency Goaltender
Takeaway
Missed Shot
Period Start
Game Official
Game End


"Goal", "Missed Shot", and "Shot" are the relevant events here.<br>
The coordinates of the "Blocked Shot" event show the position of the blocker and not the shot, so it is not useful for our analysis.<br>
Let's see the secondaryType values for our shot events.

In [0]:
display(game_plays_df.where("event IN ('Goal', 'Missed Shot', 'Shot')").select("secondaryType").distinct())

secondaryType
Wrap-around
NA
Snap Shot
Wrist Shot
Backhand
Deflected
Tip-In
Slap Shot


"Deflected" and "Tip-In" should be distinguished since they denote where the puck changed direction rather than where the source shot was taken.

We should also note that there is a shootout period type. We should disregard shots taken during a shootout, since it is under a different context than the regular play of a game.

In [0]:
display(game_plays_df.select("periodType").distinct())

periodType
SHOOTOUT
REGULAR
OVERTIME


Let's create a subset of just our shot events and relevant columns:
* play_id: Unique identifier
* event: Used to distinguish a successful shot (goal).
* secondaryType: Used to distinguish "deflected" and "tip-in" shots.
* st_x, st_y: X and Y coordinates standardized to be attacking left to right. (Note: Some coordinates are missing, so we must be sure to exclude NULL values.)

We'll use a naming convention prefix "nrm" to distinguish our dataframe as a normalized dataframe that we created, as opposed to the raw imported data.

In [0]:
nrm_shots_df = (game_plays_df
                .select("play_id", "event", "secondaryType", "st_x", "st_y")
                .where("""
                  event IN ('Goal', 'Missed Shot', 'Shot')
                  AND periodType <> 'SHOOTOUT'
                  AND st_x IS NOT NULL
                  AND st_y IS NOT NULL
                  """)
               )


To simplify our shot information, let's normalize our shot events into a boolean field "isGoal", which denotes whether the shot scored.<br>
And similarly, let's normalize our secondary information into a boolean field "isTipped", which denotes a deflected or tipped shot.

In [0]:
nrm_shots_df = (nrm_shots_df
                .withColumn("isGoal", when(col("event") == "Goal", True).otherwise(False))
                .withColumn("isTipped", when((col("secondaryType") == "Deflected") | (col("secondaryType") == "Tip-In"), True).otherwise(False))
               )


Now, let's calculate the shot angle and distance. The x and y coordinates are represented in feet from center ice, which puts the goal at (89, 0). To simplify and normalize our calculation, we will calculate the angle and distance relative to the center of the goal, disregarding the width of the net.

In [0]:
def shot_angle(x, y):
  # Nomalize position relative to the goal (89, 0) with the x-axis flipped to simplify arctan calculation.
  x_norm = (89 - x)
  angle_rad = math.atan2(y, x_norm)
  return math.degrees(angle_rad)

def shot_dist(x, y):
  # Nomalize position relative to the goal (89, 0).
  x_norm = (89 - x)
  return math.sqrt(x_norm**2 + y**2)

shot_angle_udf = udf(shot_angle, FloatType())
shot_dist_udf = udf(shot_dist, FloatType())

nrm_shots_df = (nrm_shots_df
                .withColumn("angle", shot_angle_udf(col("st_x"), col("st_y")))
                .withColumn("distance", shot_dist_udf(col("st_x"), col("st_y")))
               )


We can then group our angles and distances into categories:<br>
Angles:
* Direct: +/- 15 degrees.
* Angled: +/- 45 degrees.
* Wide: +/- 75 degrees.
* Extreme: >75 degrees.

Distances:
* Short: Within 15 feet. This is approximately the distance up to the inside of the face-off circle.
* Mid-Short: Within 30 feet. This is approximately the distance up to the face-off dot.
* Mid-Long: Within 45 feet. This is approximately the distance up to the outside of the face-off circle.
* Long: Over 45 feet.

In [0]:
def cat_angle(angle):
  if abs(angle) <= 15:
    return "Direct"
  elif abs(angle) <= 45:
    return "Angled"
  elif abs(angle) <= 75:
    return "Wide"
  else:
    return "Extreme"

def cat_dist(dist):
  if dist <= 15:
    return "Short"
  elif dist <= 30:
    return "Mid-Short"
  elif dist <= 45:
    return "Mid-Long"
  else:
    return "Long"

cat_angle_udf = udf(cat_angle, StringType())
cat_dist_udf = udf(cat_dist, StringType())

nrm_shots_df = (nrm_shots_df
                .withColumn("angle_category", cat_angle_udf(col("angle")))
                .withColumn("distance_category", cat_dist_udf(col("distance")))
                .repartition("angle_category", "distance_category")
               )

display(nrm_shots_df.orderBy("play_id"))


play_id,event,secondaryType,st_x,st_y,isGoal,isTipped,angle,distance,angle_category,distance_category
2010020001_10,Shot,Wrist Shot,81,-7,false,false,-41.185925,10.630146,Angled,Short
2010020001_102,Shot,Slap Shot,45,-23,false,false,-27.597296,49.648766,Angled,Long
2010020001_104,Missed Shot,NA,37,-21,false,false,-21.99113,56.0803,Angled,Long
2010020001_107,Shot,Snap Shot,48,-30,false,false,-36.193207,50.803543,Angled,Long
2010020001_108,Missed Shot,NA,1,1,false,false,0.6510604,88.005684,Direct,Long
2010020001_11,Shot,Snap Shot,83,-5,false,false,-39.805573,7.81025,Angled,Short
2010020001_110,Shot,Wrist Shot,58,25,false,false,38.884495,39.824615,Angled,Mid-Long
2010020001_117,Missed Shot,NA,61,29,false,false,46.005085,40.311287,Wide,Mid-Long
2010020001_118,Missed Shot,NA,57,23,false,false,35.70669,39.40812,Angled,Mid-Long
2010020001_119,Missed Shot,NA,39,-31,false,false,-31.798912,58.830265,Angled,Long


###Aggregate Shot Statistics

Now, we can create a new table of aggregated information.

In [0]:
agg_shots_df = (nrm_shots_df
                .groupBy("angle_category", "distance_category", "isTipped")
                .agg(
                  sum(col("isGoal").cast("tinyint")).alias("total_goals"),
                  count(col("isGoal")).alias("total_shot_attempts"),
                  (
                    round(
                      (
                        sum(col("isGoal").cast("float")) 
                        / count(col("isGoal"))
                      ) * 100
                      , 2)
                  ).alias("shot_percentage")
                ).orderBy(col("shot_percentage").desc())
               )
                
display(agg_shots_df)

angle_category,distance_category,isTipped,total_goals,total_shot_attempts,shot_percentage
Wide,Mid-Long,true,54,183,29.51
Extreme,Short,true,86,363,23.69
Direct,Short,true,3343,16641,20.09
Angled,Short,true,3569,18009,19.82
Extreme,Mid-Long,true,10,51,19.61
Extreme,Mid-Short,true,22,116,18.97
Wide,Short,true,647,3492,18.53
Direct,Mid-Short,true,1719,9921,17.33
Wide,Long,true,7,43,16.28
Direct,Short,false,8114,49987,16.23


###Results
We can see some skewing from categories with lower shot attempts. To note, these categories seem to be mid to long distance tipped shots, so the skewing could be attributable to how the recordkeeper records these. If a shot is tipped from far out, but doesn't go on net, how often is something like that recorded as a shot attempt or simply not recorded? For example, the highest percentage shot (but with low total shot attempts) is a wide-angled, mid-long distance tipped shot. From that distance and angle, the play was more likely a hard pass or rim-around attempt that got deflected towards the net. If it's not on net, it would unlikely be considered a shot attempt and more likely just considered an accidental deflection.

If we discount the skewed results, however, we can clearly see that short range tipped shots from any angle are the most successful. For non-tipped shots, short range direct or angled shots are also fairly successful. This simply reinforces the intuitive notion and confirms the basic strategies of many hockey coaches and analysts: you can score more if you can get in close.

#Generate Player-Influenced Win Statistics

A statistic often cited on broadcasts is how often a team wins when a particular player scores, the narrative being that certain players have a greater influence over the outcome of the game. Intuitively though, if any player scores, then their team is more likely to win. Is there any significance to this statistic?

First, let's gather relevant statistics into a normalized table.

In [0]:
nrm_player_scoring_df = (game_skater_stats_df.alias("skater")
                         .join(game_teams_stats_df.alias("teams"),
                               (col("teams.game_id") == col("skater.game_id"))
                               & (col("teams.team_id") == col("skater.team_id")),
                               "inner")
                         .join(player_info_df.alias("player"),
                               col("player.player_id") == col("skater.player_id"),
                               "inner")
                         .join(team_info_df,
                               team_info_df["team_id"] == col("teams.team_id"),
                               "inner")
                         .withColumn("player_name", concat(col("player.firstName"), lit(" "),col("player.lastName")))
                         .withColumn("points", col("skater.assists") + col("skater.goals"))
                         .select("skater.player_id", 
                                 "player_name",
                                 "player.primaryPosition",
                                 "skater.game_id", 
                                 "skater.team_id", 
                                 team_info_df["abbreviation"],
                                 "skater.assists", 
                                 "skater.goals", 
                                 "points",
                                 "teams.won")
                         .repartition("player_id")
                        )

display(nrm_player_scoring_df)


player_id,player_name,primaryPosition,game_id,team_id,abbreviation,assists,goals,points,won
8477476,Artturi Lehkonen,LW,2016020610,8,MTL,1,0,1,false
8477461,Remi Elie,LW,2017020221,25,DAL,0,0,0,false
8476458,Ryan Strome,C,2016020298,2,NYI,0,0,0,false
8476439,Brett Ritchie,RW,2017020986,25,DAL,0,1,1,false
8477461,Remi Elie,LW,2017020986,25,DAL,1,0,1,false
8476458,Ryan Strome,C,2017021124,22,EDM,1,1,2,true
8476856,Matt Dumba,D,2017020146,30,MIN,1,0,1,true
8476458,Ryan Strome,C,2016020716,2,NYI,0,0,0,true
8477476,Artturi Lehkonen,LW,2017020706,8,MTL,0,0,0,true
8476458,Ryan Strome,C,2017021076,22,EDM,0,0,0,false


###Aggregate Statistics

Now we can aggregate the statistics by player.

In [0]:
agg_player_influence_df = (nrm_player_scoring_df
                           .groupBy("player_id", "player_name", "primaryPosition")
                           .agg(
                             count(col("player_id")).alias("games_played"),
                             sum(when(col("goals") > 0, 1).otherwise(0)).alias("games_with_goal"),
                             sum(when(col("goals") == 0, 1).otherwise(0)).alias("games_without_goal"),
                             sum(when(col("points") > 0, 1).otherwise(0)).alias("games_with_point"),
                             sum(when(col("points") == 0, 1).otherwise(0)).alias("games_without_point"),
                             sum(when((col("goals") > 0) & (col("won") == True), 1).otherwise(0)).alias("wins_with_goal"),
                             sum(when((col("goals") == 0) & (col("won") == True), 1).otherwise(0)).alias("wins_without_goal"),
                             sum(when((col("points") > 0) & (col("won") == True), 1).otherwise(0)).alias("wins_with_point"),
                             sum(when((col("points") == 0) & (col("won") == True), 1).otherwise(0)).alias("wins_without_point")
                           )
                           .withColumn("win_percentage_with_goal", 
                                       when(col("games_with_goal") == 0, 0)
                                       .otherwise(
                                         round(
                                           (col("wins_with_goal").cast("float") 
                                            / col("games_with_goal")) 
                                           * 100
                                           , 2)
                                       )
                                      )
                           .withColumn("win_percentage_without_goal", 
                                       when(col("games_without_goal") == 0, 0)
                                       .otherwise(
                                         round(
                                           (col("wins_without_goal").cast("float") 
                                            / col("games_without_goal")) 
                                           * 100
                                           , 2)
                                       )
                                      )
                           .withColumn("win_percentage_with_point", 
                                       when(col("games_with_point") == 0, 0)
                                       .otherwise(
                                         round(
                                           (col("wins_with_point").cast("float") 
                                            / col("games_with_point")) 
                                           * 100
                                           , 2)
                                       )
                                      )
                           .withColumn("win_percentage_without_point", 
                                       when(col("games_without_point") == 0, 0)
                                       .otherwise(
                                         round(
                                           (col("wins_without_point").cast("float") 
                                            / col("games_without_point")) 
                                           * 100
                                           , 2)
                                       )
                                      )
                           .orderBy(col("win_percentage_with_point").desc(), col("win_percentage_with_goal").desc())
                          )

display(agg_player_influence_df)


player_id,player_name,primaryPosition,games_played,games_with_goal,games_without_goal,games_with_point,games_without_point,wins_with_goal,wins_without_goal,wins_with_point,wins_without_point,win_percentage_with_goal,win_percentage_without_goal,win_percentage_with_point,win_percentage_without_point
8475102,Travis Turnbull,C,3,1,2,1,2,1,2,1,2,100.0,100.0,100.0,100.0
8478074,Anthony Angello,C,32,4,28,4,28,4,16,4,16,100.0,57.14,100.0,57.14
8477201,Kellan Lain,LW,9,1,8,1,8,1,3,1,3,100.0,37.5,100.0,37.5
8475309,Jerry D'Amigo,RW,31,1,30,2,29,1,9,2,8,100.0,30.0,100.0,27.59
8473562,John McCarthy,C,88,3,85,6,82,3,52,6,49,100.0,61.18,100.0,59.76
8451805,Ken Sutton,D,74,1,73,8,66,1,38,8,31,100.0,52.05,100.0,46.97
8479945,Gavin Bayreuther,D,76,8,68,16,60,8,32,16,24,100.0,47.06,100.0,40.0
8477680,Josh Currie,RW,84,8,76,16,68,8,32,16,24,100.0,42.11,100.0,35.29
8471265,Raymond Sawada,RW,11,1,10,1,10,1,2,1,2,100.0,20.0,100.0,20.0
8473494,Petteri Wirtanen,C,3,1,2,1,2,1,0,1,0,100.0,0.0,100.0,0.0


###Results

Clearly, we can see that this statistic is easily skewed and has misleading results, even for players with a high number of games played. Furthermore, this statistic is often cited on broadcasts within the context of a single season, which further adds to the skewness.

For further exploration, though, let's add some thresholds and take a look at goals versus points results separately.

In [0]:
display(agg_player_influence_df
        .select("player_id", "player_name", "primaryPosition", "games_played", "games_with_goal", "games_without_goal", 
                "wins_with_goal", "wins_without_goal", "win_percentage_with_goal", "win_percentage_without_goal")
        .where("games_with_goal >= 50")
        .orderBy(col("win_percentage_with_goal").desc())
       )

display(agg_player_influence_df
        .select("player_id", "player_name", "primaryPosition", "games_played", "games_with_point", "games_without_point", 
                "wins_with_point", "wins_without_point", "win_percentage_with_point", "win_percentage_without_point")
        .where("games_with_point >= 100")
        .orderBy(col("win_percentage_with_point").desc())
       )


player_id,player_name,primaryPosition,games_played,games_with_goal,games_without_goal,wins_with_goal,wins_without_goal,win_percentage_with_goal,win_percentage_without_goal
8477450,Jason Dickinson,C,726,75,651,66,327,88.0,50.23
8477488,Brett Pesce,D,812,53,759,46,385,86.79,50.72
8480069,Cale Makar,D,328,60,268,52,152,86.67,56.72
8476854,Hampus Lindholm,D,950,75,875,65,401,86.67,45.83
8473463,Leo Komarov,C,962,92,870,79,408,85.87,46.9
8476975,Cedric Paquette,C,978,106,872,91,552,85.85,63.3
8475343,Nic Dowd,C,671,69,602,59,313,85.51,51.99
8476880,Tom Wilson,RW,1037,218,819,184,420,84.4,51.28
8479944,Zach Aston-Reese,C,454,51,403,43,225,84.31,55.83
8477964,Ivan Barbashev,C,797,108,689,91,357,84.26,51.81


player_id,player_name,primaryPosition,games_played,games_with_point,games_without_point,wins_with_point,wins_without_point,win_percentage_with_point,win_percentage_without_point
8478472,Mathieu Joseph,RW,444,108,336,92,212,85.19,63.1
8446053,Chris Chelios,D,492,103,389,87,219,84.47,56.3
8476975,Cedric Paquette,C,978,189,789,159,484,84.13,61.34
8475343,Nic Dowd,C,671,153,518,126,246,82.35,47.49
8474145,Robert Bortuzzo,D,783,114,669,93,344,81.58,51.42
8476329,Travis Boyd,C,337,113,224,92,127,81.42,56.7
8477450,Jason Dickinson,C,726,181,545,146,247,80.66,45.32
8478519,Anthony Cirelli,C,751,309,442,249,261,80.58,59.05
8479944,Zach Aston-Reese,C,454,106,348,85,183,80.19,52.59
8476927,Teddy Blueger,C,405,120,285,96,128,80.0,44.91


Now, we can see a slightly more normalized pattern, and I wonder if there is something there beyond flashy broadcast statistics. Perhaps a more informative statistic would be the differential of win percentage with a goal or point versus without.

In [0]:
agg_player_influence_df = (agg_player_influence_df
                           .withColumn("win_differential_with_goal", round(col("win_percentage_with_goal") - col("win_percentage_without_goal"), 2))
                           .withColumn("win_differential_with_point", round(col("win_percentage_with_point") - col("win_percentage_without_point"), 2))
                          )

display(agg_player_influence_df
        .select("player_id", "player_name", "primaryPosition", "games_played", "games_with_goal", "games_without_goal", 
                "wins_with_goal", "wins_without_goal", "win_percentage_with_goal", "win_percentage_without_goal", "win_differential_with_goal")
        .where("games_with_goal >= 50")
        .orderBy(col("win_differential_with_goal").desc())
       )

display(agg_player_influence_df
        .select("player_id", "player_name", "primaryPosition", "games_played", "games_with_point", "games_without_point", 
                "wins_with_point", "wins_without_point", "win_percentage_with_point", "win_percentage_without_point", "win_differential_with_point")
        .where("games_with_point >= 100")
        .orderBy(col("win_differential_with_point").desc())
       )


player_id,player_name,primaryPosition,games_played,games_with_goal,games_without_goal,wins_with_goal,wins_without_goal,win_percentage_with_goal,win_percentage_without_goal,win_differential_with_goal
8477839,Conor Sheary,LW,818,127,691,106,277,83.46,40.09,43.37
8476854,Hampus Lindholm,D,950,75,875,65,401,86.67,45.83,40.84
8475197,Tyson Barrie,D,1075,132,943,108,400,81.82,42.42,39.4
8473463,Leo Komarov,C,962,92,870,79,408,85.87,46.9,38.97
8476326,Josh Archibald,RW,590,95,495,78,219,82.11,44.24,37.87
8477450,Jason Dickinson,C,726,75,651,66,327,88.0,50.23,37.77
8477955,Jared McCann,C,763,134,629,108,275,80.6,43.72,36.88
8477498,Darnell Nurse,D,835,73,762,58,330,79.45,43.31,36.14
8477488,Brett Pesce,D,812,53,759,46,385,86.79,50.72,36.07
8480830,Andrei Svechnikov,RW,657,172,485,140,221,81.4,45.57,35.83


player_id,player_name,primaryPosition,games_played,games_with_point,games_without_point,wins_with_point,wins_without_point,win_percentage_with_point,win_percentage_without_point,win_differential_with_point
8480012,Elias Pettersson,C,624,384,240,252,52,65.63,21.67,43.96
8478474,Lawson Crouse,LW,707,188,519,143,176,76.06,33.91,42.15
8471675,Sidney Crosby,C,1471,1014,457,683,125,67.36,27.35,40.01
8477937,Jake Virtanen,RW,760,229,531,169,184,73.8,34.65,39.15
8477290,Michael Raffl,LW,900,242,658,192,265,79.34,40.27,39.07
8477934,Leon Draisaitl,C,907,624,283,362,55,58.01,19.43,38.58
8476455,Gabriel Landeskog,LW,1131,653,478,433,134,66.31,28.03,38.28
8478413,Jordan Greenway,LW,619,183,436,134,153,73.22,35.09,38.13
8479343,Clayton Keller,C,729,362,367,234,99,64.64,26.98,37.66
8480069,Cale Makar,D,328,212,116,160,44,75.47,37.93,37.54


It looks like there is less variation in the results when we look at points instead of goals, but we can see a lot of well-known star players end up sorting to the top. Perhaps there is something to this differential statistic.